## Loading Data from HAD and ESK into the Database

Hartland and Eskdalemuir data can be retrieved from [intermagnet](http://www.intermagnet.org/data-donnee/download-eng.php) in the iaga2002 format or from the [BGS world download center](http://www.wdc.bgs.ac.uk/dataportal/) as iaga2002, json or csv.
2 functions are provided here, which give an example as to how to load BGS json data or iaga2002 data to the database.

Import relevant libs:

In [8]:
import os
import io 

import json
import numpy as np
import pandas as pd
from IPython.display import display

from geomagio.iaga2002 import IAGA2002Factory
import matplotlib.pyplot as plt
%matplotlib inline

Define functions for loading iaga2002/ json data.

In [23]:
def load_bgs_json(folder):
    """
    Function to load BGS json magnetometer files into a pandas dataframe.
    
    Args:
        folder (str): Folder containing json files.
        
    Returns:
        pd.DataFrame: A pandas dataframe containing magnetometer readings.
    """
    for f in os.listdir(folder):
        if f.endswith(".json"):
            with open(os.path.join(folder, f)) as dat_file:
                mag_json = json.load(dat_file)
                try:
                    mag_dat = mag_dat.append(pd.DataFrame(mag_json["timeSeries"]["dataPoints"]))
                except NameError:
                    mag_dat = pd.DataFrame(mag_json["timeSeries"]["dataPoints"])
    
    mag_dat["time"] = pd.to_datetime(mag_dat["time"])
    mag_dat.set_index("time", inplace=True)
    mag_dat.sort_index(inplace=True)
    return mag_dat.astype("float32")


def iaga2002_to_df(file):
    row = next(file)
    while row:
        vals = row.split()
        if vals[0] == "DATE":
            vals.remove('|')
            vals.remove("DOY")
            vals.remove("DATE")
            vals[0] = vals[0].lower()
            vals[1:] = (label[-1] for label in vals[1:])
            cols = vals
            break
        row = next(file)
    
    all_rows = []
    for i, row in enumerate(file):
        vals = row.split()
        vals[0] = vals[0] + " " + vals.pop(1)
        del vals[1]
        all_rows.append(vals)
    
    mag_dat = pd.DataFrame(all_rows, columns = cols)
    mag_dat["time"] = pd.to_datetime(mag_dat["time"])
    mag_dat.set_index("time", inplace=True)
    mag_dat.sort_index(inplace=True)
    return mag_dat.astype("float32")
    

def load_intermagnet_iaga_min(folder):
    """
    Function for loading iaga minute data from a folder to a pandas dataframe.
    
    Args:
        folder (str): The directory containing iaga 2002 data.
    
    Returns:
        pd.DataFrame: Pandas dataframe containing magnetometer data.
    """
    for f in os.listdir(folder):
        # ".min" should be replaced with ".hor" to make it compatible with hourly magnetometer data.
        if f.endswith(".min"):
            with open(os.path.join(folder, f)) as dat_file:
                try:
                    mag_dat = mag_dat.append(iaga2002_to_df(dat_file))
                except NameError:
                    mag_dat = iaga2002_to_df(dat_file)
    
    mag_dat.sort_index(inplace=True)
    return mag_dat

### Load and insert HAD data.
An example of loading HAD data from iaga2002 files and BGS json files are given below:

In [25]:
# Load iaga2002 files, stored in the folder HAD_iaga.
HAD_iaga_df = load_intermagnet_iaga_min("HAD_iaga")
display(HAD_iaga_df.head())
# Load BGS json files, stored in the folder HAD_json.
HAD_json_df = load_bgs_json("HAD_json")
display(HAD_json_df.tail())

,H,D,Z,F
time,,,,
2017-01-01 00:00:00,19754.900391,-120.080002,44372.898438,48571.800781
2017-01-01 00:01:00,19754.699219,-120.000000,44372.898438,48571.699219
2017-01-01 00:02:00,19754.599609,-119.949997,44372.898438,48571.601562
2017-01-01 00:03:00,19755.000000,-119.879997,44373.199219,48572.101562
2017-01-01 00:04:00,19755.400391,-119.790001,44373.398438,48572.398438


,F,X,Y,Z
time,,,,
2016-12-31 23:55:00,48571.0,19743.0,-691.0,44372.0
2016-12-31 23:56:00,48571.0,19743.0,-691.0,44372.0
2016-12-31 23:57:00,48571.0,19743.0,-691.0,44372.0
2016-12-31 23:58:00,48571.0,19743.0,-691.0,44372.0
2016-12-31 23:59:00,48572.0,19743.0,-690.0,44373.0


Note that the json data uses XYZF coordinate scheme. In the database, We said that the sensor was using the coordinates HDZF. Therefore, convert XY to HD, as defined by the algorithm in [XYZ Algorithm](https://github.com/usgs/geomag-algorithms/blob/master/docs/algorithms/XYZ.md) . Note that D is an angle, measured in arcminutes (see https://www.ngdc.noaa.gov/IAGA/vdat/IAGA2002/iaga2002format.html ).

In [36]:
# Convert XY to HD.
HAD_json_df["H"] = np.sqrt(HAD_json_df["X"]**2 + HAD_json_df["Y"]**2)
# np.arctan returns radians, multiply by rad_to_arcmin to convert to arcminutes.
rad_to_arcmin = 180*60/np.pi
HAD_json_df["D"] = np.arctan(HAD_json_df["Y"]/HAD_json_df["X"])*rad_to_arcmin
HAD_json_df.head()

,F,X,Y,Z,H,D
time,,,,,,
2011-01-01 00:00:00,48439.0,19642.0,-1052.0,44265.0,19670.152344,-183.945511
2011-01-01 00:01:00,48439.0,19642.0,-1052.0,44265.0,19670.152344,-183.945511
2011-01-01 00:02:00,48439.0,19642.0,-1052.0,44265.0,19670.152344,-183.945511
2011-01-01 00:03:00,48439.0,19642.0,-1053.0,44265.0,19670.205078,-184.120026
2011-01-01 00:04:00,48439.0,19642.0,-1053.0,44265.0,19670.205078,-184.120026


Combine the json data and the iaga data, and load to the database.

In [38]:
# Combine dataframes.
HAD_df = HAD_iaga_df.append(HAD_json_df[["H", "D", "Z", "F"]])
display(HAD_df.head())
display(HAD_df.tail())

# Write data to the database.
from HAD_dat_insert import write_mag_dat as insert_HAD
insert_HAD(HAD_df.reset_index().to_dict("records"))

,H,D,Z,F
time,,,,
2017-01-01 00:00:00,19754.900391,-120.080002,44372.898438,48571.800781
2017-01-01 00:01:00,19754.699219,-120.000000,44372.898438,48571.699219
2017-01-01 00:02:00,19754.599609,-119.949997,44372.898438,48571.601562
2017-01-01 00:03:00,19755.000000,-119.879997,44373.199219,48572.101562
2017-01-01 00:04:00,19755.400391,-119.790001,44373.398438,48572.398438


,H,D,Z,F
time,,,,
2016-12-31 23:55:00,19755.087891,-120.271179,44372.0,48571.0
2016-12-31 23:56:00,19755.087891,-120.271179,44372.0,48571.0
2016-12-31 23:57:00,19755.087891,-120.271179,44372.0,48571.0
2016-12-31 23:58:00,19755.087891,-120.271179,44372.0,48571.0
2016-12-31 23:59:00,19755.052734,-120.097252,44373.0,48572.0


### Load and insert ESK data.

Repeat the steps above for the Eskdalemuir observatory data.

In [41]:
# Load iaga2002 files, stored in the folder ESK_iaga.
ESK_iaga_df = load_intermagnet_iaga_min("ESK_iaga")

# Load BGS json files, stored in the folder ESK_json.
ESK_json_df = load_bgs_json("ESK_json")


# Convert XY to HD.
ESK_json_df["H"] = np.sqrt(ESK_json_df["X"]**2 + ESK_json_df["Y"]**2)
# np.arctan returns radians, multiply by rad_to_arcmin to convert to arcminutes.
ESK_json_df["D"] = np.arctan(ESK_json_df["Y"]/ESK_json_df["X"])*rad_to_arcmin
ESK_json_df.head()

# Combine dataframes.
ESK_df = ESK_iaga_df.append(ESK_json_df[["H", "D", "Z", "F"]])
display(ESK_df.head())
display(ESK_df.tail())

# Write data to the database.
from ESK_dat_insert import write_mag_dat as insert_ESK
insert_ESK(ESK_df.reset_index().to_dict("records"))

,H,D,Z,F
time,,,,
2017-01-01 00:00:00,17549.699219,-139.759995,46501.500000,49702.898438
2017-01-01 00:01:00,17549.400391,-139.639999,46501.398438,49702.699219
2017-01-01 00:02:00,17549.199219,-139.570007,46501.398438,49702.601562
2017-01-01 00:03:00,17549.500000,-139.470001,46501.300781,49702.601562
2017-01-01 00:04:00,17550.000000,-139.330002,46501.199219,49702.699219


,H,D,Z,F
time,,,,
2016-12-31 23:55:00,17550.490234,-139.699142,46501.0,49703.0
2016-12-31 23:56:00,17550.490234,-139.699142,46501.0,49703.0
2016-12-31 23:57:00,17550.490234,-139.699142,46501.0,49703.0
2016-12-31 23:58:00,17550.490234,-139.699142,46501.0,49703.0
2016-12-31 23:59:00,17550.449219,-139.503418,46501.0,49703.0


Now that the data is loaded, the [server](../server), for retrieving magnetometer data, needs setting up.